In [53]:
import pickle as pkl
import numpy as np
import seaborn as sns
%matplotlib inline

sns.set(context="poster", style='whitegrid', font='sans-serif')

X_real = pkl.load(open('/data/SPRINT/X_processed.pkl', 'rb'))
y_real = pkl.load(open('/data/SPRINT/y_processed.pkl', 'rb'))
print(X_real.shape, y_real.shape)

public_dir = '/data/SPRINT/nonprivate/'
private_dir = '/data/SPRINT/private/'

((6502, 3, 12), (6502,))
